In [1]:
df = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferschema","true")\
    .load("s3://project.group23.fan5.flight/Scaled_Data_M/")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1648311854923_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.functions import col, column

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#test data on 2018 

trainDF = df.where(col("YEAR")!=2018)
testDF = df.where(col("YEAR")==2018)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
trainDF, testDF1 =  trainDF.randomSplit([0.5,0.5], seed = 143)
trainDF1, testDF =  testDF.randomSplit([0.5,0.5], seed = 143)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# print the count of observations in each set
print("Observations in training set = ", trainDF.count())
print("Observations in testing set = ", testDF.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Observations in training set =  26811048
Observations in testing set =  3539398

In [6]:
# import the RandomForestClassifier function from the pyspark.ml.classification package
from pyspark.ml.classification import RandomForestClassifier

# Build the RandomForestClassifier object 'dt' by setting the required parameters
# We will pass the VectorIndexed columns as featureCol for Random Forest. Since they can handle categorical indexes
rf_2 = RandomForestClassifier(featuresCol="scaledfeatures", labelCol="FLIGHT_STATUS" , numTrees=75, maxDepth= 7)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# fit the RandomForestClassifier object on the training data
rfmodel_2 = rf_2.fit(trainDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#This RandomForestClassifierModel can be used as a transformer to perform prediction on the testing data
rfpredictonDF_2 = rfmodel_2.transform(testDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='FLIGHT_STATUS', 
    predictionCol='prediction', 
    metricName='accuracy')
accuracy = evaluator.evaluate(rfpredictonDF_2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
print("Accuracy: ", accuracy)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy:  0.6501922078274328

In [12]:
# 5. F1 Score (F-measure)
print("F1 Score = ", evaluator.evaluate(rfpredictonDF_2, {evaluator.metricName: "f1"}))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

F1 Score =  0.5488800679604634

In [14]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

preds_and_labels = rfpredictonDF_1.select(['prediction','FLIGHT_STATUS']).withColumn('FLIGHT_STATUS', F.col('FLIGHT_STATUS').cast(FloatType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','FLIGHT_STATUS'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[2241562.   39259.]
 [1206905.   51672.]]

In [13]:
from pyspark.ml.tuning import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…